In [ ]:
install command

pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

python -m pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

pip install opencv-contrib-python-headless


In [ ]:
import json
import cv2
from tqdm import tqdm
import os
from pqdm.processes import pqdm

train = None
test = None

def rewrite(l):
    fname, ext = os.path.splitext(l)
    img_path = '/home/jovyan/data/filtered-food/'+l
    new_path = '/home/jovyan/data/filtered-food-jpg/'+fname+'.jpg'
    try:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(new_path, img)
    except:
        print(img_path)



train = !ls /home/jovyan/data/filtered-food

print('start filtered-image image rewrite')
_ = pqdm(train, rewrite, n_jobs=50)



In [1]:
temp = !ls /home/jovyan/data/filtered-food

'00000853-b7e3-480a-b8ca-09d7f2b683d5.jpg'

In [42]:
import json

all_images = !ls /home/jovyan/data/filtered-food-jpg

with open('/home/jovyan/workspace/ml_mg/json_data/163train.json', 'r') as f:
    # print(train[0])
    l_all = ""
    for l in f.readlines():
        l_all += l.strip()
    train = json.loads(l_all)

with open('/home/jovyan/workspace/ml_mg/json_data/163test.json', 'r') as f:
    # print(train[0])
    l_all = ""
    for l in f.readlines():
        l_all += l.strip()
    test = json.loads(l_all)

with open('/home/jovyan/workspace/ml_mg/json_data/163val.json', 'r') as f:
    # print(train[0])
    l_all = ""
    for l in f.readlines():
        l_all += l.strip()
    val = json.loads(l_all)

In [28]:
len(train['images']) + len(test['images']) + len(val['images'])

30061

In [52]:
import numpy as np

diff_images_names = list(set(all_images) - set(i['file_name'] for i in train['images']) - set([i['file_name'] for i in test['images']])  - set([i['file_name'] for i in val['images']]))
print(len(diff_images_names))

72491


In [ ]:
temp_train_image = train['images']
temp_train_image.sort(key=lambda x : x['id'])

for i, img in enumerate(temp_train_image):
    if str(img['id']) != str(i):
        print(i, img)

In [20]:
print(len([i['id'] for i in train['images']]))
print()

24048

In [53]:

import os
from pqdm.processes import pqdm
import cv2

diff_images = []

def get_size(l):
    img_path = '/home/jovyan/data/filtered-food-jpg/'+l
    try:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        height, width, _ = img.shape
        return {
            'file_name': l,
            'height': height,
            'width': width
        }
    except:
        print(img_path)

print('start filtered-image image rewrite')
diff_images = pqdm(diff_images_names, get_size, n_jobs=50)


start filtered-image image rewrite


SUBMITTING | : 100%|██████████| 72491/72491 [00:05<00:00, 13479.62it/s]
PROCESSING | : 100%|██████████| 72491/72491 [05:11<00:00, 232.76it/s]
COLLECTING | : 100%|██████████| 72491/72491 [00:00<00:00, 477648.86it/s]


In [37]:
print(sum([1 if x["id"] == 30061 else 0 for x in train["images"]]))
print(sum([1 if x["id"] == 30061 else 0 for x in test["images"]]))
print(sum([1 if x["id"] == 30061 else 0 for x in val["images"]]))

0
0
0


In [51]:
print(train.keys())
print(train['licenses'])
print(len(diff_images))

dict_keys(['annotations', 'categories', 'images', 'info', 'licenses'])
None
96539


In [83]:
from tqdm import tqdm

save_train = {'annotations':[], 'categories':train['categories'], 'images':[], 'info':train['info'], 'licenses':train['licenses']}

for i, img in tqdm(enumerate(diff_images)):
	img["id"] = 30061 + i
	temp_anno = {
		'area': 100,
		'bbox': [0, 0, 10, 10],
		'category_id': 2,
		'id': 200000+i,
		# 'ignore': 0,
		'image_id': 30061+i,
		'iscrowd': 0,
		# 'segmentation': [] 
		}
	save_train['images'].append(img)
	save_train['annotations'].append(temp_anno)

72491it [00:00, 445110.64it/s]


In [ ]:
python train_net.py --num-gpus 1 --config configs/custom/faster_rcnn_R_50_FPN_food.yaml  SOLVER.IMG_PER_BATCH_LABEL 8 SOLVER.IMG_PER_BATCH_UNLABEL 8

In [84]:
import json

with open('/home/jovyan/workspace/ml_mg/json_data/163train_unlabel.json', 'w') as f:


    json.dump(save_train, f)

In [87]:
import json

for x in train['annotations']:
    x['bbox'] = [float(f) for f in x['bbox']]
    # del x['segmentation']
    # del x['ignore']

with open('/home/jovyan/workspace/ml_mg/json_data/163train2.json', 'w') as f:
    json.dump(train, f)

In [ ]:
python train_net.py --num-gpus 1 --config configs/custom/faster_rcnn_R_50_FPN_food.yaml  SOLVER.IMG_PER_BATCH_LABEL 8 SOLVER.IMG_PER_BATCH_UNLABEL 8

In [69]:
val["annotations"][len(val["annotations"])-1]

{'area': 4224832,
 'bbox': [328, 440, 2104, 2008],
 'category_id': 162,
 'id': 149011,
 'ignore': 0,
 'image_id': 30049,
 'iscrowd': 0,
 'segmentation': []}

In [88]:
import json

with open('/home/jovyan/workspace/ml_mg/json_data/163train.json', 'r') as f:
    # print(train[0])
    l_all = ""
    for l in f.readlines():
        l_all += l.strip()
    t1 = json.loads(l_all)

with open('/home/jovyan/workspace/ml_mg/json_data/163train_unlabel.json', 'r') as f:
    # print(train[0])
    l_all = ""
    for l in f.readlines():
        l_all += l.strip()
    t2 = json.loads(l_all)


In [79]:
from tqdm import tqdm

t1_image_id = [x['image_id'] for x in t1['annotations']]
print(t1_image_id)
for x in tqdm(t1['images']):
    if x['id'] not in t1_image_id:
        print(x)

t2_image_id = [x['image_id'] for x in t2['annotations']]
for x in tqdm(t2['images']):
    if x['id'] not in t2_image_id:
        print(x)

100%|██████████| 72491/72491 [00:35<00:00, 2047.15it/s]
